In [239]:
import glob

import numpy as np
import pandas as pd

## Let's check the fish activity file first

In [240]:
fname = './data/fish_actvity_for_validation/150717_2A_2B.csv'

# Load in the data
df = pd.read_csv(fname)

df.head()

,location,animal,user,sn,an,datatype,start,end,startreason,endreason,frect,fredur,midct,middur,burct,burdur,stdate,sttime
0,c97,z097,ZEBRALAB02\zebralab_user,1,0,quant,0.0,60.0,Beginning of session,End of period,34,55.7,34,4.3,0,0.0,17/07/2015,14:30:00
1,c98,z098,ZEBRALAB02\zebralab_user,1,0,quant,0.0,60.0,Beginning of session,End of period,0,60.0,0,0.0,0,0.0,17/07/2015,14:30:00
2,c99,z099,ZEBRALAB02\zebralab_user,1,0,quant,0.0,60.0,Beginning of session,End of period,6,59.0,6,1.0,0,0.0,17/07/2015,14:30:00
3,c100,z100,ZEBRALAB02\zebralab_user,1,0,quant,0.0,60.0,Beginning of session,End of period,25,55.8,25,4.2,0,0.0,17/07/2015,14:30:00
4,c101,z101,ZEBRALAB02\zebralab_user,1,0,quant,0.0,60.0,Beginning of session,End of period,1,60.0,0,0.0,0,0.0,17/07/2015,14:30:00


Since we only care the fish 1-96 (which are already genotyped), we could slice out the data of those fish first!

In [241]:
# we could see the ID of the fish is bury in the animal column, so we could generate a ID column first
df['ID'] = df['animal'].str[1:]
df.head()

,location,animal,user,sn,an,datatype,start,end,startreason,endreason,frect,fredur,midct,middur,burct,burdur,stdate,sttime,ID
0,c97,z097,ZEBRALAB02\zebralab_user,1,0,quant,0.0,60.0,Beginning of session,End of period,34,55.7,34,4.3,0,0.0,17/07/2015,14:30:00,097
1,c98,z098,ZEBRALAB02\zebralab_user,1,0,quant,0.0,60.0,Beginning of session,End of period,0,60.0,0,0.0,0,0.0,17/07/2015,14:30:00,098
2,c99,z099,ZEBRALAB02\zebralab_user,1,0,quant,0.0,60.0,Beginning of session,End of period,6,59.0,6,1.0,0,0.0,17/07/2015,14:30:00,099
3,c100,z100,ZEBRALAB02\zebralab_user,1,0,quant,0.0,60.0,Beginning of session,End of period,25,55.8,25,4.2,0,0.0,17/07/2015,14:30:00,100
4,c101,z101,ZEBRALAB02\zebralab_user,1,0,quant,0.0,60.0,Beginning of session,End of period,1,60.0,0,0.0,0,0.0,17/07/2015,14:30:00,101


In [242]:
# Slice out the fish that has ID < 96
df.loc[:,'ID'] = df.loc[:, 'ID'].astype(int)
df_fish = df.loc[df['ID'] <= 96, :]
df_fish.head()

,location,animal,user,sn,an,datatype,start,end,startreason,endreason,frect,fredur,midct,middur,burct,burdur,stdate,sttime,ID
96,c1,z001,ZEBRALAB02\zebralab_user,1,0,quant,0.0,60.0,Beginning of session,End of period,0,60.0,0,0.0,0,0.0,17/07/2015,14:29:59,1
97,c2,z002,ZEBRALAB02\zebralab_user,1,0,quant,0.0,60.0,Beginning of session,End of period,10,58.5,10,1.5,0,0.0,17/07/2015,14:29:59,2
98,c3,z003,ZEBRALAB02\zebralab_user,1,0,quant,0.0,60.0,Beginning of session,End of period,17,57.6,16,2.4,0,0.0,17/07/2015,14:29:59,3
99,c4,z004,ZEBRALAB02\zebralab_user,1,0,quant,0.0,60.0,Beginning of session,End of period,4,59.5,4,0.6,0,0.0,17/07/2015,14:29:59,4
100,c5,z005,ZEBRALAB02\zebralab_user,1,0,quant,0.0,60.0,Beginning of session,End of period,38,54.9,37,5.2,0,0.0,17/07/2015,14:29:59,5


#### Check whether there are NaN in the acitivity column (middur)
#### Check whether acitivity column (middur) >=0 [because the middur is time in 1 minute]
#### Check whether acitivity column (middur) <= 60s [because the middur is time in 1 minute]
#### Check the time interval is always 1 minute across the whole session
#### Check whether each fish has the same amount of data point

In [243]:
# Check whether there are NaN in the acitivity column (middur)
def test_missing_data(df, fname):
    """Look for missing entries."""
    assert np.all(df['middur'].notnull()), fname + ' contains missing data'
    
test_missing_data(df_fish, fname)

It passed! There are no NaN in the dataset.

In [244]:
# Check whether acitivity column (middur) >=0 [because the middur is time in 1 minute]
def test_negative(df, fname):
    """Look for negative scattering values in cells."""
    assert np.all(df['middur'] >= 0), \
            fname + ' contains negative scattering data'

test_negative(df_fish,fname)

It passed! All the activity data is not negative.

In [245]:
# Check whether acitivity column (middur) <= 60s [because the middur is time in 1 minute]
def test_less_than_60s(df, fname):
    """Look for negative scattering values in gated cells."""
    assert np.all(df['middur']<= 60), \
            fname + ' contains data that large than 60s'

test_less_than_60s(df_fish,fname)

AssertionError: ./data/fish_actvity_for_validation/150717_2A_2B.csv contains data that large than 60s

Seems there are some data in the activity column (middur) is larger than 60s, which dosen't make sense when considering the definition of middur (time in one minute), let's take a look at them

In [ ]:
df_activity_exceed_60 = df_fish.loc[df['middur']> 60]
df_activity_exceed_60

We could see there are 6 datapoints in the activity column are >60, which is inconsistent with the researcher's claim. Maybe we should exclude those datapoints when doing the real analyse.  

And when we look at the time interval of this datapoint(end-start = 76.5) which is larger than 60s, that explains why the activity data is >60s. But it brings up the concern that the time for each measurement is not always 1 minutes across whole dataset, let's check that in the next step.

In [ ]:
# Generate a column of time length for each measurement using the 'end' and 'start' column
df_fish.loc['time_length',:] = df_fish['end'] - df_fish['start']
df_fish.head()

In [ ]:
# Check the time length is always 1 minute across the whole session
def test_time_length(df, fname):
    """Look for negative scattering values in gated cells."""
    assert np.all(df['time_length']!= 60), \
            fname + ' contains time length that is not 60s'

test_time_length(df_fish,fname)

It didn't pass, indicating not all time length is 60s which is consistent with our observation in the last step. Let's take a look at those measurement in time >60s.

In [246]:
df_fish.loc[df_fish['time_length']!= 60]

KeyError: 'time_length'

We could see that there are actually three measurements of activity is recorded in a session >60s. We need to keep that in mind when doing the data analyse. 

We would also like to know whether each fish has same anounts of data points.

In [247]:
# check whether each fish has the same amounts of datapoints
df_datapoints_num = pd.DataFrame(df_fish.groupby('ID').size())
df_datapoints_num.head()

,0
ID,
1,4119
2,4119
3,4119
4,4119
5,4119


In [248]:
df_datapoints_num.columns = ['datapoints_num']
df_datapoints_num.loc[df_datapoints_num['datapoints_num'] != 4119]

,datapoints_num
ID,


All the fish has 4119 datapoints.

#### In general, the major problems we see in the behavior dataset are:
#### 1.The data is not evenly distributed in time, the time length of 3 measurements are >60s while all the other measurements are recorded within a minute.
#### 2.The first problem leads to the observation that 5 datapoints in the 'middur' column are >60s, which is inconsistent with the researcher's definition for 'middur'
#### When we do the real abalysis on this dataset, it's better to exclude these datapoints or normalize them to their bin size (time_length)

## Let's also check the genotype data

The researcher said that, 'the fish in instrument 2A, numbered 1 through 96 in the activity data file 150717_2A_2B.csv, were genotyped'. That indicates the genotype data we get here should has 96 fish's genotype. So let's load the file and have a look.

In [249]:
# Load in the genotype file, call it df_gt for genotype DataFrame
df_gt = pd.read_csv('./data/fish_actvity_for_validation/150717_2A_genotype_3.txt',
                    delimiter='\t',
                    comment='#')

# Take a look at it
df_gt

,wt,het,mut
0,1.0,5,2.0
1,3.0,6,4.0
2,8.0,7,14.0
3,12.0,9,16.0
4,21.0,10,26.0
5,28.0,13,27.0
6,31.0,15,37.0
7,33.0,17,40.0
8,34.0,18,41.0
9,35.0,19,46.0


In [250]:
# Tidy the DataFrame
df_gt = pd.melt(df_gt, var_name='genotype', value_name='location')

# Take a look
df_gt

,genotype,location
0,wt,1.0
1,wt,3.0
2,wt,8.0
3,wt,12.0
4,wt,21.0
5,wt,28.0
6,wt,31.0
7,wt,33.0
8,wt,34.0
9,wt,35.0


In [251]:
# Drop all the NaN and sort the dataset based on 'location'
df_gt = df_gt.dropna()
df_gt = df_gt.sort_values(['location'])
df_gt

,genotype,location
0,wt,1.0
76,mut,2.0
1,wt,3.0
77,mut,4.0
38,het,5.0
39,het,6.0
40,het,7.0
2,wt,8.0
41,het,9.0
42,het,10.0


After we drop the NaN, we see there are only 75 rows of genotype data, which is inconsistent with the researcher's claim. So we would like to check out what happens to the data: 
#### 1. We would like to see which location is not genotyped and we would rule out that data when analyze behavior.
#### 2. We would also make sure each location only reports single genotype.

In [252]:
# Check the location that is not genotyped by looking at the difference of location between 2 conseceutive row
df_gt['location+1'] = df_gt['location'].shift(-1)
df_gt['location_difference'] = df_gt['location+1'] - df_gt['location']
df_gt

,genotype,location,location+1,location_difference
0,wt,1.0,2.0,1.0
76,mut,2.0,3.0,1.0
1,wt,3.0,4.0,1.0
77,mut,4.0,5.0,1.0
38,het,5.0,6.0,1.0
39,het,6.0,7.0,1.0
40,het,7.0,8.0,1.0
2,wt,8.0,9.0,1.0
41,het,9.0,10.0,1.0
42,het,10.0,12.0,2.0


In [253]:
# if some locations are skipped, we would see the location interval >1
df_gt.loc[df_gt['location_difference']>1]

,genotype,location,location+1,location_difference
42,het,10.0,12.0,2.0
4,wt,21.0,23.0,2.0
9,wt,35.0,37.0,2.0
85,mut,46.0,48.0,2.0
59,het,50.0,52.0,2.0
86,mut,54.0,56.0,2.0
15,wt,61.0,63.0,2.0
63,het,63.0,71.0,8.0
95,mut,82.0,85.0,3.0
74,het,94.0,96.0,2.0


From the above data, we could see that 11,22,36,47,51,55,62,64-70,83,84,95 are not genotyped. 
It would be better for them to have a fourth column clarified that these fish haven't be genotyped in the original genotype file.

In [254]:
# We would also like to make sure each location reports single genotype, we would see the location interval !=0
df_gt.loc[df_gt['location_difference']==0]

,genotype,location,location+1,location_difference


There is nothing here, which helps us make sure that single location reports single genotype

#### In general, the major problems we see in the genotype dataset are:
#### Location 11,22,36,47,51,55,62,64-70,83,84,95 are not genotyped, we need to exclude those wells wehn performing the data analysis.
#### It would be better for them to have a fourth column in their genotype file clarifying the ones that were not genotyped.

In [255]:
%load_ext watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [256]:
%watermark -v -p numpy,pandas,glob,jupyterlab

CPython 3.6.6
IPython 7.0.1

numpy 1.15.2
pandas 0.23.4
glob n
jupyterlab 0.35.0
